# 04. QLoRA SFT — 양자화 후 성능 회복

**전략:** 양자화로 떨어진 정확도를 QLoRA fine-tuning으로 회복.  
이게 상위권을 진짜로 노릴 수 있는 차별화 포인트.

**파이프라인:**  
```
EXAONE-4.0-1.2B (원본 BF16)
    ↓ [이 노트북]
QLoRA SFT (4-bit bitsandbytes 로드 + LoRA 학습)
    ↓
LoRA 가중치 merge → BF16 풀모델
    ↓ [01 또는 02 노트북]
GPTQ/FP8 양자화 → 제출
```

**환경:** Google Colab Free (T4 16GB)  
⚠️ T4 16GB에서는 QLoRA rank를 작게 유지해야 메모리 안전

## 1. 패키지 설치

> ⚠️ 설치 후 **런타임 재시작** 필요

In [ ]:
!pip install -q \
    "transformers>=4.54.0" \
    peft \
    trl \
    bitsandbytes \
    datasets \
    accelerate \
    sentencepiece \
    llmcompressor \
    "compressed-tensors==0.13.0"

print("\n✅ 설치 완료! 런타임을 재시작하세요")

## 2. 환경 확인

In [ ]:
import torch
print(f"GPU : {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

import transformers, peft, trl
print(f"transformers: {transformers.__version__}")
print(f"peft        : {peft.__version__}")
print(f"trl         : {trl.__version__}")

## 3. 설정값

> T4 16GB 기준 안전한 설정. 메모리 부족 시 `LORA_RANK` 또는 `BATCH_SIZE` 줄일 것.

In [ ]:
# ============================================================
# QLoRA SFT 설정
# ============================================================

MODEL_ID     = "LGAI-EXAONE/EXAONE-4.0-1.2B"
SFT_OUT_DIR  = "./model_sft_lora"      # LoRA adapter 저장
MERGED_DIR   = "./model_sft_merged"    # LoRA merge된 풀모델
QUANT_OUT_DIR = "./model"              # 최종 양자화 결과 (제출용)

# 데이터셋: MANTA-1M (평가 태스크와 aligned)
DATASET_ID    = "LGAI-EXAONE/MANTA-1M"
DATASET_SPLIT = "train"
NUM_TRAIN_SAMPLES = 2000   # T4에서 현실적인 학습량
MAX_SEQ_LENGTH    = 512    # T4 16GB 메모리 안전 범위

# LoRA 설정 (T4 16GB 최적화)
LORA_RANK        = 8       # 메모리 부족 시 4로 줄이기
LORA_ALPHA       = 16      # 보통 rank * 2
LORA_DROPOUT     = 0.05
LORA_TARGET      = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"]  # 모든 Linear 타겟

# 학습 설정
BATCH_SIZE       = 1       # T4에서 안전한 배치 크기
GRAD_ACCUM       = 8       # 효과적 배치 = 1 * 8 = 8
LEARNING_RATE    = 2e-4
NUM_EPOCHS       = 1
WARMUP_RATIO     = 0.03

ZIP_NAME = "submit_sft_gptq"

print("=" * 50)
print(f"모델         : {MODEL_ID}")
print(f"학습 샘플    : {NUM_TRAIN_SAMPLES}")
print(f"LoRA rank    : {LORA_RANK}")
print(f"학습률       : {LEARNING_RATE}")
print(f"Effective BS : {BATCH_SIZE * GRAD_ACCUM}")
print("=" * 50)

## 4. 모델 로드 (4-bit bitsandbytes로 메모리 절약)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 4-bit bitsandbytes 설정 (학습 중 메모리 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,   # 이중 양자화로 추가 메모리 절약
    bnb_4bit_quant_type="nf4",        # NF4: QLoRA 논문 권장
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print("토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("모델 로드 중 (4-bit bitsandbytes)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# gradient checkpointing: 메모리 절약 (속도는 약간 느려짐)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

print(f"\n로드 후 GPU 메모리: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
print("✅ 모델 로드 완료")

## 5. LoRA 어댑터 설정

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# k-bit 학습 준비 (bitsandbytes 4-bit 모델용)
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 확인
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"학습 파라미터: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")
print("✅ LoRA 설정 완료")

## 6. 학습 데이터 준비

In [ ]:
from datasets import load_dataset

print(f"학습 데이터 로드: {NUM_TRAIN_SAMPLES}샘플")
ds = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{NUM_TRAIN_SAMPLES}]")

def preprocess(example):
    """SFT를 위한 전처리: full conversation을 하나의 텍스트로"""
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=False,  # SFT는 False (응답까지 포함)
            tokenize=False,
        )
    }

ds = ds.map(preprocess, remove_columns=ds.column_names)

print(f"✅ 데이터 준비: {len(ds)}개")
print(f"샘플 미리보기:\n{ds[0]['text'][:300]}...")

## 7. SFT 학습

> T4에서 2000샘플 1 epoch ≈ 20~40분

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
import os

training_args = TrainingArguments(
    output_dir=SFT_OUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",       # 메모리 효율 optimizer
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,
    logging_steps=10,
    save_steps=500,
    fp16=False,
    bf16=True,
    dataloader_num_workers=0,
    report_to="none",               # wandb 등 외부 로깅 비활성화
    remove_unused_columns=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds,
    args=training_args,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
    packing=False,
)

print("SFT 학습 시작!")
trainer.train()
print("\n✅ SFT 학습 완료")

## 8. LoRA Merge → BF16 풀모델 저장

In [ ]:
import os, gc
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 메모리 정리
del trainer
gc.collect()
torch.cuda.empty_cache()

print("LoRA adapter 저장 중...")
model.save_pretrained(SFT_OUT_DIR)
tokenizer.save_pretrained(SFT_OUT_DIR)

print("\nLoRA merge를 위해 원본 BF16 모델 재로드 중...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

print("LoRA merge 중...")
peft_model = PeftModel.from_pretrained(base_model, SFT_OUT_DIR)
merged_model = peft_model.merge_and_unload()  # LoRA를 base에 통합

print(f"merge된 모델 저장 중: {MERGED_DIR}")
os.makedirs(MERGED_DIR, exist_ok=True)
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("\n✅ LoRA merge 완료!")
print(f"   → 다음 단계: {MERGED_DIR} 를 01 또는 02 노트북에서 양자화")

## 9. SFT Merge 모델 양자화 (GPTQ W4A16)

merge된 BF16 모델을 바로 이어서 양자화

In [ ]:
# 메모리 정리
del merged_model, peft_model, base_model
gc.collect()
torch.cuda.empty_cache()
print(f"GPU 메모리 정리 완료: {torch.cuda.memory_allocated()/1024**3:.2f} GB 사용 중")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
import time

# SFT merge된 모델로 양자화
QUANT_SOURCE = MERGED_DIR   # SFT merge 결과
QUANT_CALIB_SAMPLES = 512
QUANT_MAX_LEN = 1024

print(f"SFT merge 모델 로드: {QUANT_SOURCE}")
tokenizer_q = AutoTokenizer.from_pretrained(QUANT_SOURCE, trust_remote_code=True)
model_q = AutoModelForCausalLM.from_pretrained(
    QUANT_SOURCE,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# 캘리브레이션 데이터
ds_q = load_dataset(DATASET_ID, split=f"{DATASET_SPLIT}[:{QUANT_CALIB_SAMPLES}]")
ds_q = ds_q.map(lambda ex: {"text": tokenizer_q.apply_chat_template(
    ex["conversations"], add_generation_prompt=True, tokenize=False)})

# GPTQ 양자화
print("GPTQ 양자화 시작...")
start = time.time()
recipe = [GPTQModifier(scheme="W4A16", targets=["Linear"], ignore=["embed_tokens", "lm_head"])]
oneshot(model=model_q, dataset=ds_q, recipe=recipe,
        max_seq_length=QUANT_MAX_LEN, num_calibration_samples=QUANT_CALIB_SAMPLES)

print(f"양자화 완료 ({(time.time()-start)/60:.1f}분)")

# 저장
import os, shutil
from pathlib import Path
os.makedirs(QUANT_OUT_DIR, exist_ok=True)
model_q.save_pretrained(QUANT_OUT_DIR, save_compressed=True)
tokenizer_q.save_pretrained(QUANT_OUT_DIR)

# 제출 zip
shutil.make_archive(base_name=ZIP_NAME, format="zip", root_dir=".", base_dir=QUANT_OUT_DIR)
zip_size = Path(f"{ZIP_NAME}.zip").stat().st_size / 1024**3
print(f"\n✅ 완료! {ZIP_NAME}.zip ({zip_size:.2f} GB)")
print("   → Dacon 제출 페이지에 업로드하세요")

---
## 📊 SFT 실험 기록

| 실험 | 학습 샘플 | Rank | Epoch | 이후 양자화 | LB Score | 비고 |
|------|----------|------|-------|------------|----------|------|
| SFT-1 | 2000 | 8 | 1 | GPTQ W4A16 | - | 기본 설정 |

---
## ⚠️ 메모리 부족 시 대처법

1. `LORA_RANK` 를 8 → 4 로 줄이기
2. `NUM_TRAIN_SAMPLES` 를 2000 → 1000 으로 줄이기
3. `MAX_SEQ_LENGTH` 를 512 → 256 으로 줄이기
4. Colab Pro로 업그레이드 (A100 사용 가능)